In [12]:
%load_ext autoreload
%autoreload 2
#2 means reload everytime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import tensorflow as tf
import pandas as pd
import numpy as np
import lhapdf
import matplotlib.pyplot as plt

import functions_new

In [14]:
class DataANN(object):
    def __init__(self, pdfset='cteq61',
                 ff_PIp='NNFF10_PIp_nlo', ff_PIm='NNFF10_PIm_nlo', ff_PIsum='NNFF10_PIsum_nlo',
                 ff_KAp='NNFF10_KAp_nlo', ff_KAm='NNFF10_KAm_nlo'):
        '''
        Get data in proper format for neural network
        '''
        self.pdfData = lhapdf.mkPDF(pdfset)
        self.ffDataPIp = lhapdf.mkPDF(ff_PIp, 0)
        self.ffDataPIm = lhapdf.mkPDF(ff_PIm, 0)
        self.ffDataPIsum = lhapdf.mkPDF(ff_PIsum, 0)
        self.ffDataKAp = lhapdf.mkPDF(ff_KAp, 0)
        self.ffDataKAm = lhapdf.mkPDF(ff_KAm, 0)
        # needs to be extended to generalize for kaons
        self.eu = 2/3
        self.eubar = -2/3
        self.ed = -1/3
        self.edbar = 1/3
        self.es = -1/3
        self.esbar = 1/3
        
        self.ffDict = {0: self.ffDataPIp,
                       1: self.ffDataPIm,
                       2: self.ffDataPIsum,
                       3: self.ffDataKAp,
                       4: self.ffDataKAm}
    

    def pdf(self, flavor, x, QQ):
        return np.array([self.pdfData.xfxQ2(flavor, ax, qq) for ax, qq in zip(x, QQ)])
    
    
    def ff(self, func, flavor, z, QQ):
        return np.array([func.xfxQ2(flavor, az, qq) for az, qq in zip(z, QQ)])    
    

    def makeData(self, df, hadrons, dependencies):
        
        data = {'x': [],
             'z': [],
             'phT': [],
             'uexpr': [],
             'ubarexpr': [],
             'dexpr': [],
             'dbarexpr': [],
             'sexpr': [],
             'sbarexpr': []}
        
        y = []
        err = []
        
        df = df.loc[df['hadron'].isin(hadrons), :]
        df = df.loc[df['1D_dependence'].isin(dependencies), :]
        #X = np.array(df[['x', 'z', 'phT', 'Q2', 'hadron']])
        for i, had in enumerate(['pi+', 'pi-', 'pi0', 'k+', 'k-']):
            sliced = df.loc[df['hadron'] == had, :]
            y += list(sliced['Siv'])
            err += list(sliced['tot_err'])
            
            x = sliced['x']
            z = sliced['z']
            QQ = sliced['Q2']
            data['uexpr'] += list(self.eu**2 * self.pdf(2, x, QQ) * self.ff(self.ffDict[i], 2, z, QQ))
            data['ubarexpr'] += list(self.eubar**2 * self.pdf(-2, x, QQ) * self.ff(self.ffDict[i], -2, z, QQ))
            data['dexpr'] += list(self.ed**2 * self.pdf(1, x, QQ) * self.ff(self.ffDict[i], 1, z, QQ))
            data['dbarexpr'] += list(self.edbar**2 * self.pdf(-1, x, QQ) * self.ff(self.ffDict[i], -1, z, QQ))
            data['sexpr'] += list(self.es**2 * self.pdf(3, x, QQ) * self.ff(self.ffDict[i], 3, z, QQ))
            data['sbarexpr'] += list(self.esbar**2 * self.pdf(-3, x, QQ) * self.ff(self.ffDict[i], -3, z, QQ))

            data['x'] += list(x)
            data['z'] += list(z)
            data['phT'] += list(sliced['phT'])
        
        for key in data.keys():
            data[key] = np.array(data[key])
        
        print(data)
        return data, np.array(y), np.array(err)                      
       

In [40]:
import copy

def Create_Data(num_points,m1):
    """
    num_points will be the data points for each hadron
    
    """
    data_dictionary={"hadron":[],"x":[],"z":[],"phT":[],"Q2":[],"Siv":[],"tot_err":[]}
    data_dictionary["x"]=np.random.uniform(0.01,0.9,size=num_points)
    data_dictionary["z"]=np.random.uniform(0.2,0.5,size=num_points)
    data_dictionary["phT"]=np.random.uniform(0.3,0.9,size=num_points)
    data_dictionary["Q2"]=np.random.uniform(1,10,size=num_points)
    PiP=copy.deepcopy(data_dictionary)
    PiM=copy.deepcopy(data_dictionary)
    Pi0=copy.deepcopy(data_dictionary)
    KP=copy.deepcopy(data_dictionary)
    KM=copy.deepcopy(data_dictionary)
    PiPclass=functions_new.PiPlus()
    PiMclass=functions_new.PiMinus()
    Pi0class=functions_new.PiZero()
    KPclass=functions_new.KPlus()
    KMclass=functions_new.KMinus()
    for i in range(num_points):
        temp=np.array([[data_dictionary["x"][i],data_dictionary["z"][i],
                        data_dictionary["phT"][i],data_dictionary["Q2"][i]]])
        #########################
        PiP["hadron"].append("pi+")
        PiP["Siv"].append(PiPclass.sivers(temp,m1)[0])
        PiP["tot_err"].append(PiP["Siv"][i]*0.05)
        #########################
        PiM["hadron"].append("pi-")
        PiM["Siv"].append(PiMclass.sivers(temp,m1)[0])
        PiM["tot_err"].append((PiM["Siv"][i]*0.05))
        #########################
        Pi0["hadron"].append("pi0")
        Pi0["Siv"].append(Pi0class.sivers(temp,m1)[0])
        Pi0["tot_err"].append((Pi0["Siv"][i]*0.05))
        #########################
        KP["hadron"].append("k+")
        KP["Siv"].append(KPclass.sivers(temp,m1)[0])
        KP["tot_err"].append((KP["Siv"][i]*0.05))
        #########################
        KM["hadron"].append("k-")
        KM["Siv"].append(KMclass.sivers(temp,m1)[0])
        KM["tot_err"].append((KM["Siv"][i]*0.05))
        #########################
    #raise exception
    PiPdf=pd.DataFrame(PiP)
    PiMdf=pd.DataFrame(PiM)
    Pi0df=pd.DataFrame(Pi0)
    KPdf=pd.DataFrame(KP)
    KMdf=pd.DataFrame(KM)
    totdf=pd.concat([PiPdf,PiMdf,Pi0df,KPdf,KMdf])
    return totdf

In [41]:
test=Create_Data(50,1)

In [42]:
test

,hadron,x,z,phT,Q2,Siv,tot_err
0,pi+,0.513526,0.442095,0.684909,1.339854,0.254584,0.012729
1,pi+,0.616187,0.371346,0.673570,9.024172,0.312903,0.015645
2,pi+,0.082545,0.412879,0.300251,2.333048,0.001965,0.000098
3,pi+,0.296058,0.309651,0.348845,9.211695,0.024672,0.001234
4,pi+,0.744508,0.336301,0.452405,8.110570,0.230431,0.011522
...,...,...,...,...,...,...,...
45,k-,0.574155,0.488223,0.790171,9.989838,0.446787,0.022339
46,k-,0.638751,0.266983,0.662343,5.433715,0.234746,0.011737
47,k-,0.295076,0.229091,0.897303,6.389218,0.069034,0.003452
48,k-,0.168636,0.289835,0.457059,2.589836,0.010543,0.000527
